IMPORTS

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

BASES

In [2]:
# LIEN ACTEURS

# df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
# df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
# df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

MERGES

In [3]:
# CHANGEMENTS DANS BASE TITLE EPISODES

df_title_episodes = pd.read_csv('BD/title.episode.tsv.gz', compression = 'gzip', sep = '\t')

df_title_episodes[['seasonNumber', 'episodeNumber']] = df_title_episodes[['seasonNumber', 'episodeNumber']].replace('\\N', 0)
df_title_episodes[['seasonNumber', 'episodeNumber']] = df_title_episodes[['seasonNumber', 'episodeNumber']].apply(pd.to_numeric)

df_serie = df_title_episodes.groupby('parentTconst').sum()
df_serie = df_serie[['seasonNumber', 'episodeNumber']]
df_serie = df_serie.reset_index()

df_episodes = pd.DataFrame(df_title_episodes['tconst'])

df_serie['titleType'] = 'serie'
df_serie = df_serie.rename(columns = {'parentTconst': 'tconst'})
df_episodes['titleType'] = 'episode'
df_serie = df_serie.rename(columns = {'parentTconst': 'tconst'})
df_title_episodes = pd.concat([df_serie, df_episodes])

df_serie, df_episodes = 0, 0

In [4]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14651/3795572048.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [5]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

df_title_ratings, df_title_basics = 0, 0

In [6]:
df2 = pd.merge(df1,
               df_title_episodes,
               how = 'outer',
               on = 'tconst')

df1, df_title_episodes = 0, 0

In [7]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

df_3 = pd.merge(df2,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

df2, df_title_akas = 0, 0

In [8]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

df_tmbd_full, df_3 = 0, 0

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14651/2029903383.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


CLEAN DU MERGE

In [9]:
# DROP CE QUI N'EST PAS FILM OU SERIE

df_films = df_films[(df_films['titleType_x'] == 'movie') | (df_films['titleType_x'] == 'tvEpisode') | (df_films['titleType_x'] == 'tvSeries') | (df_films['titleType_y'] == 'serie') | (df_films['titleType_y'] == 'episode')]

In [10]:
# REPLACE \\N

df_films = df_films.replace('\\N', None)

In [11]:
# COLONNES DATES

df_films['release_date'] = pd.to_datetime(df_films['release_date']).dt.year

In [12]:
# COLONNES NUMERIQUES

df_films[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0, inplace = True)
df_films[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14651/3552025123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_films[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0, inplace = True)


In [14]:
# RELEASE YEAR

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [17]:
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

ValueError: Unable to parse string "nan" at position 0

In [ ]:
for n in range(len(df_films)):
    if df_films['year_exact'].iloc[n] != '0':
        df_films['Decennie'].iloc[0] = df_films['year_exact'].iloc[0].str[:3] + "0's"

In [ ]:
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])

In [ ]:
# RUNTIME & RATINGS

df_films['difference_runtime'] = abs(df_films['runtime'] - df_films['runtimeMinutes'])
df_films['runtime_exact'] = ''
df_films['difference_vote'] = abs(df_films['averageRating'] - df_films['vote_average'])
df_films['vote_exact'] == ''

In [ ]:
for n in range(len(df_films)):
    if df_films['difference_runtime'].iloc[n] == 0:
        df_films['runtime_exact'].iloc[n] = df_films['runtime'].iloc[n]
    else:
        df_films['runtime_exact'].iloc[n] = df_films[['runtimeMinutes', 'runtime']].max(axis = 1)  

In [ ]:
for n in range(len(df_films)):
    if df_films['difference_vote'].iloc[n] == 0:
        df_films['vote_exact'].iloc[n] = df_films['vote_average'].iloc[n]
    elif df_films['difference_vote'].iloc[n] <= 1:
        df_films['vote_exact'].iloc[n] = (df_films['averageRating'].iloc[n] + df_films['vote_average'].iloc[n]) / 2
        df_films['vote_exact'].iloc[n] = df_films[['averageRating', 'vote_average']].iloc[n].mean(axis = 1)
    else:
        df_films['vote_exact'].iloc[n] = ((df_films['vote_count'].iloc[n] * df_films['vote_average'].iloc[n]) + (df_films['numVotes'].iloc[n] * df_films['averageRating'].iloc[n])) / (df_films['numVotes'].iloc[n] + df_films['vote_count'].iloc[n])
        df_films['vote_exact'].iloc[n] = round(df_films['vote_exact'].iloc[n], 1) 

In [ ]:
# GENRES

df_films['total_genres'] = df_films['genres_x'] + df_films['genres_y']

EXPORT

In [ ]:
# df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')